## AGENTS from very ground up 


(Code for Computer Vision Africa - Workshop)

Author: Tariq Jamil

(TensorDot Solutions - www.tensordotsolutions.com)

Linkedin: https://www.linkedin.com/in/tj-yazman/

Dated: 26. October, 2024

In [28]:
from IPython.display import Image, display
import re

from Tools_r3 import calculate, currency_converter, get_news, ddg_search, get_weather
from Tools_r3 import cprint
from jinja2 import Template

#from Models_r1 import GroqChatLLM
#llm = GroqChatLLM(model='llama3-70b-8192')
#def call_llm1(message):
#    resp = llm.generate_response(messages=message)
#    return resp

In [29]:
import os
import openai
from groq import Groq

client = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY"),
)

def call_llm(message, client=client):

    if not isinstance(message, list): # in case formatted message is not given
        messages=[
            # Set an optional system message. This sets the behavior of the
            {"role": "system", "content": "you are a helpful assistant."},
            # Set a user message for the assistant to respond to.
            {"role": "user", "content": message}
            ]
    else:
        messages=message
    
    chat_completion = client.chat.completions.create(
        messages=messages,            
        model="llama-3.2-90b-text-preview",
        #model='llama3-70b-8192',
        temperature=0,
        max_tokens=128,
        stream=False,
    )

    return chat_completion.choices[0].message.content

### The system prompt can be broken down into 3 portions for understanfing sake.
1. System prompt -- persona, system_message (ReAct methodology)
2. Tools Ingestion (Tools description, and parameters they take)
3. Example to boost LLM chain of thought

#### System prompt

In [30]:
name = "Assistant"
role = "A general purpose assistant capable of answering user questions"

system_prompt_template = """
You are an intelligent '{{ name }}' and you act as '{{ role }}':

{% if tools %}
Actions you can use:
{% for tool_name, tool_func in tools.items() %}
{{ tool_name }}:
Usage: {{ tool_func.__doc__.strip() }}
{% endfor %}
{% endif %}

You run in a loop of Thought, Action, PAUSE, Observation. 
At the end of the loop, you output an Answer.

Use Thought to reason about the question you have been asked. Break up long questions as necessary. 
Use Action to run one action at a time available to you, then return PAUSE. 
Observation will be the result of running that action. 
Reflect on the observation to refine your thoughts for the next iteration.

{% if cot_example %}
Chain of Thought Example(s): 
'''{{ cot_example }}'''
{% endif %}

GO!
"""

#### Chain of Thought Example

In [31]:
example =\
"""
Question: What is the weather in New York?
Thought: I should use get_weather to find the current weather in New York.
Action: get_weather: "New York"
PAUSE

You will be called again with this:

Observation: The current weather in New York is 75°F with clear skies.

You then output:
Answer: The current weather in New York is 75°F with clear skies.
"""

### Testing system prompt

In [32]:
tools = {'ddg_search': ddg_search, 'get_news': get_news, 'get_weather': get_weather, "calculate": calculate}
system_prompt = Template(system_prompt_template).render(name=name, role=role, tools=tools, cot_example=example)

print(system_prompt)


You are an intelligent 'Assistant' and you act as 'A general purpose assistant capable of answering user questions':


Actions you can use:

ddg_search:
Usage: Searches the web for a query using DuckDuckGo and returns the results.

get_news:
Usage: Retrieves the latest news based on a specified topic using DuckDuckGo.

get_weather:
Usage: Retrieves the current weather for a specified location.

calculate:
Usage: Evaluates a mathematical expression using sympy and returns the result as a float.



You run in a loop of Thought, Action, PAUSE, Observation. 
At the end of the loop, you output an Answer.

Use Thought to reason about the question you have been asked. Break up long questions as necessary. 
Use Action to run one action at a time available to you, then return PAUSE. 
Observation will be the result of running that action. 
Reflect on the observation to refine your thoughts for the next iteration.


Chain of Thought Example(s): 
'''
Question: What is the weather in New York?
Tho

#### For Agents we need to maintain a state of messages from begining till last until answer is evaluated.

The Chat Models has a dictionary style prompt, have 'role' keys and 'content' keys.

We need to maintain state between Agent cycles of Thought, Action, Pause, Observation 

### Agent Loop

In [33]:
# testing
resp = call_llm('what is capital of Nigeria and 3 major cities?')
print(resp)

The capital of Nigeria is Abuja. 

Three major cities in Nigeria are:

1. Lagos: It's the largest city in Nigeria and a major economic hub.
2. Kano: Located in the north, Kano is a significant commercial center and a major city in Nigeria.
3. Port Harcourt: Located in the south, Port Harcourt is a major city and a significant oil-producing center in Nigeria.


In [34]:
def summarize_observation(observation):
    prompt = """
    Summarize the following observation:
    {{ observation }}
    """
    prompt = Template(prompt).render(observation=observation)
    return call_llm(prompt)


### STATE & Other Initializations

In [35]:
# initializations
max_iterations = 20

# Initializing state
state = [{"role": "system", "content": system_prompt}]

#user_query = "What are top 5 news from capital of Nigeria. Also sum the numbers in '245323'"
user_query = "What is the current temperature (celius) at capital of Nigeria."
user_query1 = "What is rain condition Lagos, with time and date."
user_query2 = "What is capital of Nigeria and 3 major cities?, in bullets form"
user_query3 = "what is todays top 4 news items in Nigeria, in bullets form"
user_query4 = "What are 3 best supermarkets in Abuja, in bullets form"

In [36]:
# Initializing state with user query
next_prompt = user_query+user_query1+"calculate the (square root of 3) multiplied with (exp of 4) and add 5. Give reply in bullets"

# Iterating through the loop
for i in range(max_iterations):
    cprint(f'\nAgent <iter:{i}>:\n')
    
    state.append({'role': 'user', 'content': next_prompt}) # update state
    result = call_llm(state)
    
    state.append({'role': 'assistant', 'content': result}) # update state
    
    if "Action" in result and "PAUSE" in result and "Observation" in result:
        result = result.split("Observation:")[0]
        
    cprint(f'{result}\n')
    
    # Original regex pattern to capture action commands
    action_re = re.compile(r"Action:\s*(\w+):\s*\"([^\"]+)\"")

    match = action_re.search(result)  # Finding the first occurrence of an action
    if match:
        # Extracting tool name and parameters
        tool_name, parameters = match.groups()
        print(f"-- Running {tool_name} with parameters: {parameters} --")

        # Checking if the tool exists
        if tool_name in tools:
            try:
                # Calling the tool with the parameter string
                result_tool = tools[tool_name](parameters)
                observation = f"Observation: {result_tool}"
                cprint(observation)
            except Exception as e:
                observation = f"Observation: Failed to execute tool '{tool_name}'. Error: {str(e)}"
        else:
            observation = f"Observation: Desired Tool '{tool_name}' not found. Available tools"
    else:
        if "Answer" in result:
            break  # Exiting the loop if "Answer" is in the response
        else:
            observation = summarize_observation(result)

    next_prompt = observation


Agent <iter:0>:
Thought: To answer this question, I need to break it down into three parts. 
1. Find the current temperature in Celsius at the capital of Nigeria (Abuja).
2. Find the rain condition in Lagos with time and date.
3. Calculate the mathematical expression (square root of 3) * (exp of 4) + 5.

Action: get_weather: "Abuja"
PAUSE


-- Running get_weather with parameters: Abuja --
 -> get_weather Tool Called --

*
***
Observation: {"location": {"name": "Abuja", "region": "Federal Capital Territory", "country": "Nigeria", "lat": 9.1758, "lon": 7.1808, "tz_id": "Africa/Lagos", "localtime_epoch": 1729947042, "localtime": "2024-10-26 13:50"}, "current": {"last_updated_epoch": 1729946700, "last_updated": "2024-10-26 13:45", "temp_c": 32.2, "temp_f": 90.0, "is_day": 1, "condition": {"text": "Partly cloudy", "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png", "code": 1003}, "wind_mph": 3.1, "wind_kph": 5.0, "wind_degree": 246, "wind_dir": "WSW", "pressure_mb": 1012.0, "pressure